# Sketching Unconstrained Least Squares

Now that it seems the sketching methods are working, we are in a position to test how the count sketch transform can be used to speed up the solving of a constrained regression problem.  To begin with, we will focus on the unconstrained case and measure how the time varies across using diferent data sizes as well as the density of the data.

In [1]:
import numpy as np
import scipy as sp
from countSketch import countSketch_elt_stream 
from srht import srht_transform
from time import time
import itertools
import numba
from numba import jit

import matplotlib.pyplot as plt
%matplotlib inline

from numpy.linalg import norm

In [7]:
def iterative_hessian_general(data, targets, sketch_size,
                      num_iters):
    '''
    INPUT:
    data - n x d matrix
    targets - n x 1 target variables
    num_iters - how many iterations to perform.  
    Need log(1/eps) -- > num_iters for eps accuracy where
    eps is relative error in the semi-norm ||u-v||_A = 
    1/sqrt(n)*||A(u-v)||_2.
    
    OUTPUT:
    x0 - vector which approximately recovers the true solution to 
    the constrained problem
    
    
    
    TO DO:
    Add functionality for lower bound on sketch size
    '''
    
    A = data
    #tidy_data =  sort_row_order(A)
    
    y = targets
    n,d = A.shape
    x0 = np.zeros(shape=(d,1))
    m = int(sketch_size) # sketching dimension
    
    ATy = A.T@y[:,None]
    covariance_mat = A.T.dot(A)
    #print("A^Ty shape: {}".format(ATy.shape))
    #print("covariance shape: {}".format(covariance_mat.shape))
   
    for n_iter in range(int(num_iters)):
        S_A = countSketch_elt_stream(A, sketch_size)
        
        true_norm = np.linalg.norm(A@x0)**2
        approx_norm = np.linalg.norm(S_A@x0)**2
        B = S_A.T.dot(S_A)
        z = ATy + np.dot(S_A.T, np.dot(S_A,x0))- covariance_mat@x0 #
        x_new = np.linalg.lstsq(B,z)[0]
        x0 = x_new # # need to convert to vector format as 
                           # the sparse solver is different output
    return np.ravel(x0)

The following function allows for the generation of sparse data similar to the `sklearn` method but with the added flexibility of being able to set sparsity as a parameter.

In [8]:
from scipy import sparse
from sklearn.utils import shuffle

def generate_sparse_data(n_samples, n_features, density,
                         n_targets=1, bias = 0.0, tail_strength=0.5,
                        noise=0.0, permute=True, coef=True,
                        random_state=None):
    '''
    Generate a random regression problem with a sparse design matrix.
    Follow the setup from sklearn.datasets.make_regression except 
    with sparse matrices and a density parameter.
    The output is generate by applying a random linear regression
    model with n_features regressors to the generated input and 
    adding some gaussian noise with adjustable scale.
    
    
    Parameters
    ----------
    n_samples : int
        The number of samples.
    n_features : int
        The number of features.
    density : float in (0,1)
        density of the data to be generated.
    n_targets : int, optional (default=1)
        The number of regression targets, i.e., the dimension of the y output
        vector associated with a sample. By default, the output is a scalar.
    bias : float, optional (default=0.0)
        The bias term in the underlying linear model.
    tail_strength : float between 0.0 and 1.0, optional (default=0.5)
        The relative importance of the fat noisy tail of the singular values
        profile if `effective_rank` is not None.
    noise : float, optional (default=0.0)
        The standard deviation of the gaussian noise applied to the output.
    shuffle : boolean, optional (default=True)
        Shuffle the samples and the features.
    coef : boolean, optional (default=True)
        If True, the coefficients of the underlying linear model are returned.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    Returns
    -------
    X : SciPy sparse matrix of shape [n_samples, n_features].
        Note that it may often be required to use as a NumPy 
        ndarray so use .toarray() on the output.
        e.g removing any zero rows. Defines the input samples.
    y : array of shape [n_samples] or [n_samples, n_targets]
        The output values.
    coef : array of shape [n_features] or [n_features, n_targets], optional
        The coefficient of the underlying linear model. It is returned only if
        coef is True.'''
    random_state = np.random.seed(random_state)
    X = sparse.random(m=n_samples, n=n_features,density=density,
                     random_state=random_state)
    
    # In future can add n_informative to extend data generation
    ground_truth = np.zeros((n_features, n_targets))
    ground_truth = np.random.rand(n_features,n_targets)
    
    y = X@ground_truth + bias
    
    # Add noise
    if noise > 0.0:
        y += random.normal(scale=noise, size=y.shape)
        
    # Randomly permute samples and features
    if permute:
        X,y = shuffle(X,y, random_state=random_state)
        indices = np.arange(n_features)
        np.random.shuffle(indices)
        X[:,:] = X[:,indices]
        ground_truth = ground_truth[indices]
    y = np.squeeze(y)
    X = X.toarray()
    if coef:
        return X,y, np.squeeze(ground_truth)
    else:
        return X, y
    

Now set up a parameter grid which will define instances of varying parameters over size, dimension, density, and sketch size.

In [16]:
param_grid = {
    'rows' : [10000, 25000, 50000],
    'columns' : [10, 100, 250],
    'density' : [0.01],#, 0.1, 0.25],#, 0.5, 1.0],
    'sketch type' : ['CWT', 'None', ],# 'SRHT']
}
param_grid

{'columns': [10, 100, 250],
 'density': [0.01],
 'rows': [10000, 25000, 50000],
 'sketch type': ['CWT', 'None']}

In [17]:
# Setup independent variables


# Experimental output
experiment_output = {}

In [30]:
for n_rows, n_cols, density, method in itertools.product(param_grid['rows'],
                                                         param_grid['columns'],
                                                         param_grid['density'],
                                                         param_grid['sketch type']):
    
    sketch_size = 10*n_cols
    print("(n,d): ({},{}), density: {}, Method {}".format(n_rows, n_cols, density, method))
    print("Sketch size: {}".format(sketch_size))
    
    ### Generate data
    data, target, coef = generate_sparse_data(n_rows, n_cols, density)
    
    if method == 'None':
        experiment_output['None'] = {}
        # Include a loop here to determine number of samples to take
        start = time()
        x_true = np.linalg.lstsq(data, target)[0]
        solve_time = time() - start
        print("System Solve time: {}".format(solve_time))
       
    if method == "CWT":
        start = time()
        x_sketch = iterative_hessian_general(data, target, sketch_size, num_iters=10)
        solve_time = time() - start
        print("IHS CWT Solve time: {}".format(solve_time))
        
    #print("Error: {}".format(np.linalg.norm(data@(x_true - x_cwt))**2))
    
    del data, target, coef# so don't keep lots of test matrices in memory

(n,d): (10000,10), density: 0.01, Method CWT
Sketch size: 100
IHS CWT Solve time: 0.013244152069091797
(n,d): (10000,10), density: 0.01, Method None
Sketch size: 100
System Solve time: 0.0013375282287597656
(n,d): (10000,100), density: 0.01, Method CWT
Sketch size: 1000


/anaconda/envs/py35/lib/python3.5/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:55: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:17: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


IHS CWT Solve time: 0.05049538612365723
(n,d): (10000,100), density: 0.01, Method None
Sketch size: 1000
System Solve time: 0.02021336555480957
(n,d): (10000,250), density: 0.01, Method CWT
Sketch size: 2500
IHS CWT Solve time: 0.22769927978515625
(n,d): (10000,250), density: 0.01, Method None
Sketch size: 2500
System Solve time: 0.09374475479125977
(n,d): (25000,10), density: 0.01, Method CWT
Sketch size: 100
IHS CWT Solve time: 0.018318653106689453
(n,d): (25000,10), density: 0.01, Method None
Sketch size: 100
System Solve time: 0.0025517940521240234
(n,d): (25000,100), density: 0.01, Method CWT
Sketch size: 1000
IHS CWT Solve time: 0.07598543167114258
(n,d): (25000,100), density: 0.01, Method None
Sketch size: 1000
System Solve time: 0.04527854919433594
(n,d): (25000,250), density: 0.01, Method CWT
Sketch size: 2500
IHS CWT Solve time: 0.3624725341796875
(n,d): (25000,250), density: 0.01, Method None
Sketch size: 2500
System Solve time: 0.1752917766571045
(n,d): (50000,10), density:

In [28]:
import line_profiler
%load_ext line_profiler

In [31]:
data, target, coef = generate_sparse_data(50000,10,density=0.01)
%lprun -f iterative_hessian_general iterative_hessian_general(data,target,100,num_iters=10)

/anaconda/envs/py35/lib/python3.5/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:55: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [33]:
%timeit iterative_hessian_general(data,target,100,num_iters=10)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:55: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


32.7 ms ± 402 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit np.linalg.lstsq(data, target)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:257: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


4.81 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit countSketch.countSketch()